In [1]:
from pyscal_rdf.network.network import OntologyNetwork

In [2]:
#prov = OntologyNetwork('../pyscal_rdf/data/prov.rdf', delimiter='#')
cmso = OntologyNetwork('../pyscal_rdf/data/cmso.owl')
pldo = OntologyNetwork('../pyscal_rdf/data/pldo.owl')
podo = OntologyNetwork('../pyscal_rdf/data/podo.owl')
msmo = OntologyNetwork('../pyscal_rdf/data/msmo.owl')

In [4]:
combo = cmso + pldo + podo + msmo

In [5]:
combo.namespaces

{'cmso': 'http://purls.helmholtz-metadaten.de/cmso/',
 'qudt': 'http://qudt.org/schema/qudt/',
 'pldo': 'http://purls.helmholtz-metadaten.de/pldo/',
 'podo': 'http://purls.helmholtz-metadaten.de/podo/',
 'msmo': 'http://purls.helmholtz-metadaten.de/msmo/'}

Manually add provo attributes

In [6]:
combo.attributes[]

{'class': {'cmso:SimulationCell': cmso:SimulationCell,
  'cmso:UnitCell': cmso:UnitCell,
  'cmso:Angle': cmso:Angle,
  'cmso:LatticeAngle': cmso:LatticeAngle,
  'cmso:SimulationCellAngle': cmso:SimulationCellAngle,
  'cmso:AtomicScaleSample': cmso:AtomicScaleSample,
  'cmso:Atom': cmso:Atom,
  'cmso:Basis': cmso:Basis,
  'cmso:ComputationalSample': cmso:ComputationalSample,
  'cmso:MacroscaleSample': cmso:MacroscaleSample,
  'cmso:MesoscaleSample': cmso:MesoscaleSample,
  'cmso:MicroscaleSample': cmso:MicroscaleSample,
  'cmso:NanoscaleSample': cmso:NanoscaleSample,
  'cmso:CalculatedProperty': cmso:CalculatedProperty,
  'cmso:CrystallineMaterial': cmso:CrystallineMaterial,
  'cmso:CrystalDefect': cmso:CrystalDefect,
  'cmso:ChemicalSpecies': cmso:ChemicalSpecies,
  'cmso:Molecule': cmso:Molecule,
  'cmso:ChemicalElement': cmso:ChemicalElement,
  'cmso:LatticeParameter': cmso:LatticeParameter,
  'cmso:Length': cmso:Length,
  'cmso:SimulationCellLength': cmso:SimulationCellLength,
  'cm

In [16]:
combo.add_path(('cmso:Material', 'cmso:hasDefect', 'pldo:PlanarDefect'))
combo.add_path(('cmso:Material', 'cmso:hasDefect', 'podo:Vacancy'))
combo.add_path(('cmso:SimulationCell', 'podo:hasVacancyConcentration', 'float'))
combo.add_path(('cmso:SimulationCell', 'podo:hasNumberOfVacancies', 'int'))
combo.add_path(('cmso:ComputationalSample', 'prov:wasDerivedFrom', 'cmso:ComputationalSample'))
combo.add_path(('cmso:ComputationalSample', 'prov:wasGeneratedBy', 'msmo:ComputationalMethod'))
combo.add_path(('msmo:ComputationalMethod', 'prov:wasAssociatedWith', 'prov:SoftwareAgent'))
#combo.add_path(('prov:SoftwareAgent', ))

Test a complex workflow query

In [22]:
s = combo.create_query(combo.terms.cmso.ComputationalSample, [combo.terms.msmo.ComputationalMethod, combo.terms.prov.SoftwareAgent])
print(s)

PREFIX cmso: <http://purls.helmholtz-metadaten.de/cmso/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX pldo: <http://purls.helmholtz-metadaten.de/pldo/>
PREFIX podo: <http://purls.helmholtz-metadaten.de/podo/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX msmo: <http://purls.helmholtz-metadaten.de/msmo/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
SELECT DISTINCT ?ComputationalMethod ?SoftwareAgent
WHERE {
    ?ComputationalSample prov:wasGeneratedBy ?ComputationalMethod .
    ?ComputationalSample prov:wasDerivedFrom ?Bundle .
    ?Bundle prov:wasInfluencedBy ?SoftwareAgent .
FILTER 
}


Test a query

In [17]:
condition = (combo.terms.pldo.hasSigmaValue > 5)

In [18]:
s = combo.create_query(combo.terms.cmso.ComputationalSample, 
                   [combo.terms.pldo.hasSigmaValue])
print(s)

PREFIX cmso: <http://purls.helmholtz-metadaten.de/cmso/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX pldo: <http://purls.helmholtz-metadaten.de/pldo/>
PREFIX podo: <http://purls.helmholtz-metadaten.de/podo/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX msmo: <http://purls.helmholtz-metadaten.de/msmo/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
SELECT DISTINCT ?hasSigmaValuevalue
WHERE {
    ?ComputationalSample cmso:hasMaterial ?AmorphousMaterial .
    ?AmorphousMaterial cmso:hasDefect ?AntiphaseBoundary .
    ?AntiphaseBoundary pldo:hasSigmaValue ?hasSigmaValuevalue .
FILTER 
}


In [ ]:
q = cmso.create_query('cmso:ComputationalSample', ['pldo:hasSigmaValuevalue', 'cmso:hasAngle_alphavalue'], values = ['val > 5 and val < 10'])
print(q)